In [6]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
# tf.debugging.set_log_device_placement(False)

logdir = './logs/linreg'
writer = tf.summary.create_file_writer(logdir)

step = 20
slope = 0.4
bias = 1.5
rows = 100


x_train = tf.random.uniform(shape=(rows,))
perturb = tf.random.normal(shape=(len(x_train),), stddev=0.01)
y_train = slope * x_train + bias + perturb

m = tf.Variable(0.)
b = tf.Variable(0.)

def predict_y_value(x):
    y = m * x + b
    return y

@tf.function
def squared_error(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

learning_rate = 0.05
steps = 400

# Bracket the loss function call - squared_error() - with
# tf.summary.trace_on() ans tf.summary.trace_export()
tf.summary.trace_on(graph=True, profiler=False)

with writer.as_default():
    for epoch in range(steps):
        with tf.GradientTape() as tape:
            predictions = predict_y_value(x_train)
            loss = squared_error(predictions, y_train)
            tf.summary.scalar('loss', loss, step=epoch)

            gradients = tape.gradient(loss, [m, b])
            m.assign_sub(gradients[0] * learning_rate)
            b.assign_sub(gradients[1] * learning_rate)

    tf.summary.trace_export(name='mse_loss',
                            step=0,
                            profiler_outdir=logdir)
    

print("m: %f, b: %f" % (m.numpy(), b.numpy()) )


TensorFlow version:  2.4.1
m: 0.416172, b: 1.491044


In [7]:
%load_ext tensorboard
%tensorboard --logdir ./logs/linreg

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 31200), started 0:08:42 ago. (Use '!kill 31200' to kill it.)